# RF Chain and Correlator Dynamic Range Commissioning Test

## Table of Contents\n
#### [0.1 - Global Configuration](#02---setup-test-equipment)
#### [0.2 - Setup Test Equipment](#02---setup-test-equipment)
#### [1 -> Test Steps](#1---test-steps)
This notebook provides for indepenent control of the test equipment during the rf chain and correlator linearity test so that test equipment setup may be changed while the telescope is scanning.

## 0.1 Global Configuration

### 0.1.1 Import dependencies

In [ ]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from datetime import datetime
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
# from ska_oso_pdm.entities.common.target import (
#     CrossScanParameters,
#     FivePointParameters,
#     RasterParameters,
#     SinglePointParameters,
#     StarRasterParameters,
# )
# from ska_oso_pdm.entities.sdp import BeamMapping
# from ska_oso_scripting import oda_helper

# from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
# from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root

# from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
# from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 0.1.2 Setup Global Variables and Configuration

In [ ]:
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

### 0.1.3 Test Connections to Namespaces

In [ ]:
test_equipment.smoke_test()

## 0.2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 0.2.1 Configure Test Equipment State 

In [ ]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### 0.2.2 Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

### 0.2.3 Create Test Equipment Plot

In [ ]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

### 0.2.4 Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

### 0.2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

**Step 2:**

Set CW generator to -30dBm, and wideband noise at approximately -142dBm/Hz. Set the CW tone frequency to exactly at the centre of a spectral channel closest to 700MHz.

Configure a subarray to record a Spectral Line imaging product in Band 1

Expected Result:  
*Test equipment configured*

In [ ]:
# test equipment setup values - band 1
frequency_to_set = 699.99719002e6
spfrx_power_level = -30.0
sky_sim_loss = 35.0
attenuation = 20
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator
siggen_power_level = spfrx_power_level + sky_sim_loss

# print current
# print log time
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time                               :", timestr)
print(f"Current signal generator frequency                  : {signal_generator.frequency}")
print(f"Current signal generator power level                : {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band                          : {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", siggen_power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(1.5)
# print updated values and confirm updates
# print log time
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time                               :", timestr)
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(
    "Updated spfrx input power level:                      :",
    signal_generator.power_dbm - sky_sim_loss,
)
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == siggen_power_level, print(
    f"Power level required is {siggen_power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

In [ ]:
# print current only
signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator


# print current
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time                               :", timestr)
print(f"Current signal generator frequency                  : {signal_generator.frequency}")
print(f"Current signal generator power level                : {signal_generator.power_dbm}")
print(
    "Current spfrx input power level:                       ",
    (signal_generator.power_dbm - sky_sim_loss),
)
print(f"Current Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band                          : {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

**Step 4:**

Reduce the CW power level in ~3 dB steps, dwell for a time interval as close as possible to 3*the averaging time. Repeat this 20 times for a total reduction of 60 dB. 

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
spfrx_power_level = -30.0
siggen_power_level = spfrx_power_level + sky_sim_loss
for i in range(1, 22):
    signal_generator = test_equipment.signal_generator
    signal_generator.write_attribute("power_dbm", siggen_power_level)
    time.sleep(
        4.2
    )  # should be 3 x averaging time (0.14s) but 1.2 is about the minimum sig gen time
    timestr = datetime.now().strftime("%F %T.%f")[:-3]
    print("Current Date and Time                               :", timestr)
    print(
        timestr,
        f"Updated signal generator power level                  : {signal_generator.power_dbm},  spfrx input power level:",
        (signal_generator.power_dbm - sky_sim_loss),
    )
    siggen_power_level = siggen_power_level - 3.0

**Step 9:**

Repeat #1 - #7 for Band 2, with CW frequency at approximately 1300MHz.

Expected Result:  
*P_max - P_min > 45+6dB*

*Figures of kX_i & kY_i over frequency*
*RXS123 unit 1, Band 2 |kX_i| = k +/-5%*
R*XS123 unit 1, Band 2 |kY_i| = k +/-5%*

In [ ]:
# test equipment setup values - band 2
frequency_to_set = 1300.0e6  # tbc
spfrx_power_level = -30.0
sky_sim_loss = 35.0
attenuation = 20
band = 2

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator
siggen_power_level = spfrx_power_level + sky_sim_loss

# print current
# print log time
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time                               : ", timestr)
print(f"Current signal generator frequency                  : {signal_generator.frequency}")
print(f"Current signal generator power level                : {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band                          : {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", siggen_power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(1.5)
# print updated values and confirm updates
# print log time
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time                               : ", timestr)
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(
    "Updated spfrx input power level                      : ",
    signal_generator.power_dbm - sky_sim_loss,
)
print(f"Updated Sky Simulator Correlated Noise Source      : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == siggen_power_level, print(
    f"Power level required is {siggen_power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

In [ ]:
spfrx_power_level = -30.0
siggen_power_level = spfrx_power_level + sky_sim_loss
for i in range(1, 22):
    signal_generator = test_equipment.signal_generator
    signal_generator.write_attribute("power_dbm", siggen_power_level)
    time.sleep(
        4.2
    )  # should be 3 x averaging time (0.14s) but 1.2 is about the minimum sig gen time
    timestr = datetime.now().strftime("%F %T.%f")[:-3]
    print(
        timestr,
        f"Updated signal generator power level:      {signal_generator.power_dbm},  spfrx input power level:",
        (signal_generator.power_dbm - sky_sim_loss),
    )
    siggen_power_level = siggen_power_level - 3.0

**Step 10:**

Repeat #1 - #7 for Band 2, with CW frequency at approximately 1300MHz. 1355 MHz

Expected Result:  
*Test equipment configured*

In [ ]:
# test equipment setup values - band 2
frequency_to_set = 1355.0e6
spfrx_power_level = -30.0
attenuation = 10
band = 2
siggen_power_level = spfrx_power_level + sky_sim_loss
# print current
# print log time
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time: ", timestr)
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", siggen_power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(1.5)
# print updated values and confirm updates
# print log time
timestr = datetime.now().strftime("%F %T.%f")[:-3]
print("Current Date and Time                              :", timestr)
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(
    "Updated spfrx input power level:                    :",
    signal_generator.power_dbm - sky_sim_loss,
)
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == siggen_power_level, print(
    f"Power level required is {siggen_power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

In [ ]:
spfrx_power_level = -30.0
siggen_power_level = spfrx_power_level + sky_sim_loss
for i in range(1, 22):
    signal_generator = test_equipment.signal_generator
    signal_generator.write_attribute("power_dbm", siggen_power_level)
    time.sleep(
        1.2
    )  # should be 3 x averaging time (0.14s) but 1.2 is about the minimum sig gen time
    timestr = datetime.now().strftime("%F %T.%f")[:-3]
    print("Current Date and Time                              :", timestr)
    print(
        timestr,
        f"Updated signal generator power level:      {signal_generator.power_dbm},  spfrx input power level:",
        (signal_generator.power_dbm - sky_sim_loss),
    )
    siggen_power_level = siggen_power_level - 3.0